In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jumananadir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Genism and Spacy Imports

In [34]:
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim 
%matplotlib inline
import re

In [35]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [36]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [37]:
df = pd.read_csv('/Users/jumananadir/Downloads/Eluvio_DS_Challenge.csv')

In [38]:
df.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,category
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews


In [8]:
df = df[(df['date_created'] > '2008-01-24') & (df['date_created'] <= '2008-12-31')]

In [9]:
data = df.title.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Scores killed in Pakistan clashes']


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['scores', 'killed', 'in', 'pakistan', 'clashes']]


In [11]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

In [12]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
print(trigram_mod[bigram_mod[data_words[0]]])

['scores', 'killed', 'in', 'pakistan', 'clashes']


In [14]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [15]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [16]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [17]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
data_words_nostops = remove_stopwords(data_words)

In [19]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [20]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [21]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['score', 'kill', 'clash']]


In [22]:
id2word = corpora.Dictionary(data_lemmatized)

In [23]:
texts = data_lemmatized

In [24]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [25]:
id2word[0]

'clash'

In [26]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('clash', 1), ('kill', 1), ('score', 1)]]

In [27]:
#LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
# Printing the Keyword in the 10 Topics/Labels
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.186*"crisis" + 0.118*"hour" + 0.114*"reject" + 0.054*"pass" + '
  '0.014*"plunge" + 0.001*"resolution" + 0.000*"likely" + 0.000*"cease_fire" + '
  '0.000*"girl" + 0.000*"saudi"'),
 (1,
  '0.275*"year" + 0.162*"global" + 0.129*"get" + 0.071*"tell" + '
  '0.057*"support" + 0.033*"launch" + 0.031*"worker" + 0.012*"control" + '
  '0.007*"row" + 0.006*"migrant"'),
 (2,
  '0.414*"israeli" + 0.157*"child" + 0.107*"use" + 0.045*"continue" + '
  '0.021*"mass" + 0.017*"base" + 0.000*"palestinian" + 0.000*"day" + '
  '0.000*"medium" + 0.000*"terrorist"'),
 (3,
  '0.177*"government" + 0.134*"time" + 0.079*"pay" + 0.063*"violence" + '
  '0.060*"top" + 0.053*"charge" + 0.045*"old" + 0.036*"large" + '
  '0.028*"citizen" + 0.027*"ally"'),
 (4,
  '0.226*"take" + 0.100*"report" + 0.085*"still" + 0.084*"good" + 0.072*"need" '
  '+ 0.054*"run" + 0.039*"also" + 0.023*"eat" + 0.023*"website" + '
  '0.015*"poor"'),
 (5,
  '0.153*"hit" + 0.137*"dead" + 0.110*"live" + 0.075*"last" + 0.074*"spy" + '
 

In [29]:
#Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Calculate Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -24.37361000809039

Coherence Score:  0.4283470457415275


In [30]:
# Visualize the Labels
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/jumananadir/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8      0.094395  0.106606       1        1  8.433787
3      0.023826  0.013469       2        1  7.855826
15     0.274904 -0.257724       3        1  7.560037
6      0.098975  0.118647       4        1  7.414954
16    -0.030087 -0.014326       5        1  6.843576
1      0.124843  0.216967       6        1  6.829334
4     -0.027140 -0.013375       7        1  6.246184
14    -0.058378 -0.023399       8        1  6.003327
18    -0.030656 -0.014756       9        1  5.889541
2      0.007325  0.002894      10        1  5.070370
7      0.079015  0.071190      11        1  5.070015
9     -0.034694 -0.016087      12        1  4.893479
12    -0.031333 -0.014747      13        1  4.838132
13    -0.054271 -0.022238      14        1  4.176125
5     -0.070093 -0.025935      15        1  4.140031
10    -0.072015 -0.026163      16        1  2.693258
19    -0.074752 -0.025982      17        1  2.083455
17    -0.074561 -0.025759      18        1  2.059362
0     -0.074622 -0.025803      19        1  1.495219
11    -0.070680 -0.023477      20        1  0.403988, topic_info=    Category         Freq       Term        Total  loglift  logprob
70   Default  3370.000000        say  3370.000000  30.0000  30.0000
124  Default  2369.000000     attack  2369.000000  29.0000  29.0000
123  Default  2242.000000    israeli  2242.000000  28.0000  28.0000
1    Default  1922.000000       kill  1922.000000  27.0000  27.0000
69   Default  1862.000000         go  1862.000000  26.0000  26.0000
..       ...          ...        ...          ...      ...      ...
345  Topic20     0.033774   official     2.722561   1.1219  -9.4542
492  Topic20     0.033773        die     2.722407   1.1219  -9.4542
535  Topic20     0.033771  pakistani     2.722225   1.1220  -9.4542
469  Topic20     0.033771      iraqi     2.722331   1.1219  -9.4542
659  Topic20     0.033772     police     2.723870   1.1214  -9.4542

[805 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
126       3  0.986999     access
191      14  0.992916      allow
102       2  0.989084       ally
150       7  0.990156       also
155       4  0.987721        arm
...     ...       ...        ...
115       4  0.986938       work
46        6  0.986680     worker
180       8  0.998859      world
47        6  0.998841       year
213       9  0.984354  yesterday

[176 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 16, 7, 17, 2, 5, 15, 19, 3, 8, 10, 13, 14, 6, 11, 20, 18, 1, 12])